In [228]:
import cv2 
import numpy as np

In [229]:
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import pandas as pd

class LKTracker:
    def __init__(self, id):
        self.params = dict(winSize = (15, 15), maxLevel = 4, criteria = 
                           (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        self.id = id
        self.crop = None        
        self.old_points = None
        self.color = None
        self.width = None
        self.height = None
        self.active = False
        
    def track(self, old_gray, curr_gray):     
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, curr_gray, self.old_points, None, **self.params)
        return new_points  
    
    def set_crop(self, crop):
        self.crop = crop
        self.width = crop.shape[0] // 2
        self.height = crop.shape[1] // 2

def calculate_similarity(im1, im2):    
    img1 = im1.copy()
    img2 = im2.copy()
    
    img2.resize(img1.size, Image.ANTIALIAS)
    img1.resize(img2.size, Image.ANTIALIAS)
    return(ssim(img1, img2, multichannel = True))

def count_objects(frame, count, winname):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2

    frame = cv2.putText(frame, 'Number of objects:'+str(count), org, font,
                        fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow(winname,frame)
    
def detect_face_and_add_trackers(face_cascade, binary, trackers):
    global face_id
    detected_faces = face_cascade.detectMultiScale(binary)
        
    crops = []

    for (column, row, width, height) in detected_faces:              
        crop = binary[ row : row + height, column : column + width]

        crops.append(crop)

        if (crop.shape[0] > 5 and crop.shape[1] > 5):            
            if len(trackers) == 0:
                face_id = face_id + 1
                tracker = LKTracker(face_id)
                tracker.set_crop(crop)
                tracker.old_points = np.float32([[column  + (width // 1.5), row + (height // 1.5)]])
                tracker.color = colors.pop(0)
                tracker.active = True
                trackers.append(tracker)
            else:
                similarities = []
                for tracker in trackers:                    
                    similarities.append(calculate_similarity(tracker.crop, crop))

                if max(similarities) < similarity_threshold:
                    face_id = face_id + 1
                    tracker = LKTracker(face_id)
                    tracker.set_crop(crop)
                    tracker.old_points = np.float32([[column  + (width // 1), row + (height // 1)]])
                    tracker.color = colors.pop(0)
                    tracker.active = True
                    trackers.append(tracker)
    return trackers

def calculate_metric(metric_cal, frame_count, tracker, x, y):
    metric_cal = metric_cal.append({'frame': frame_count,'id' : tracker.id, 
                           'bl': x,'bt' : y,'bw': tracker.width,'bh': tracker.height,
                           'conf' : -1,'x' : -1, 'y' : -1, 'z' : -1}, ignore_index=True)
    return metric_cal

def save_file(metric_cal):
    numpy_array = metric_cal.to_numpy()
    np.savetxt("PETS09-S2L1.txt", numpy_array, fmt = "%d",  delimiter=',')

In [230]:
import numpy as np
import cv2
import sys
from scipy.spatial import distance

# resize dimensions can be changed here
imagewidth = 300
imageheight = 400
dim = (imagewidth, imageheight)

# The default capture device is the default video source.
cap = None
cap = cv2.VideoCapture('./PETS09-S2L1.mp4')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")

# Create a window
winname = "Faces"
cv2.namedWindow(winname, cv2.WINDOW_NORMAL)

hasframe, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
ret,old_binary = cv2.threshold(gray,90,255,cv2.THRESH_BINARY)  

face_id = 0
similarity_threshold = 0.25
# similarity_threshold = 0.26

# Create some random colors
colors = list(np.random.randint(0,255,(100,3)))

colors = [tuple(x) for x in colors]

trackers = []
frame_count = 1

metric_cal = pd.DataFrame(columns = ['frame','id','bl','bt','bw','bh', 'conf','x', 'y', 'z'])

while(hasframe):    
    hasframe, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret,binary = cv2.threshold(gray,90,255,cv2.THRESH_BINARY)    
 
    if frame_count % 1 == 0:
        trackers = detect_face_and_add_trackers(face_cascade, binary, trackers)
    
    for tracker in trackers:        
        new_point = tracker.track(old_binary, binary)  
        x,y = new_point.ravel()                 
        tracker.old_points = new_point
        x,y = int(x), int(y)

        color = ( int (tracker.color [ 0 ]), int (tracker.color [ 1 ]), int (tracker.color [ 2 ]))                     
        cv2.rectangle(frame, (x - tracker.height, y - tracker.width), (x + tracker.height, y + tracker.width), tuple(color), 2)
        
        metric_cal = calculate_metric(metric_cal, frame_count, tracker, x, y)
    
    if frame_count % 100 == 0:
        save_file(metric_cal)
    
    count_objects(frame, len(trackers), winname)
    cv2.imshow(winname, frame)
    
    old_binary = binary
       
    key = cv2.waitKey(60)
    if key == 27:
        break
    frame_count = frame_count + 1
    
save_file(metric_cal)
cap.release()
cv2.destroyAllWindows()

In [227]:
metric_cal

,frame,id,bl,bt,bw,bh,conf,x,y,z
0,1,1,516,194,34,17,-1,-1,-1,-1
1,2,1,515,196,34,17,-1,-1,-1,-1
2,3,1,512,199,34,17,-1,-1,-1,-1
3,4,1,512,197,34,17,-1,-1,-1,-1
4,5,1,509,199,34,17,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
5505,427,17,702,174,36,18,-1,-1,-1,-1
5506,427,18,894,358,33,16,-1,-1,-1,-1
5507,427,19,48,142,15,7,-1,-1,-1,-1
5508,427,20,866,306,37,18,-1,-1,-1,-1


In [186]:
numpy_array = metric_cal.to_numpy()
np.savetxt("test_file.txt", numpy_array, fmt = "%d",  delimiter=',')

In [170]:
# def reinforce_crops(crops, trackers):    
#     t_return = []
#     for tracker in trackers:
#         similarity = []
        
#         for crop in crops:
#             similarity.append(calculate_similarity(tracker.crop, crop))
            
#         tracker.set_crop(crops[similarity.index(max(similarity))])
#         t_return.append(tracker)
#     return t_return

In [ ]:
# #         print('times_drawn')
#         print('coords',x, y)    
#         cv2.rectangle(frame, (y, x), (y + 10, x + 10), tracker.color, 4)

In [207]:
aa = 2

def ff(aa):
    aa = aa + 1

In [208]:
aa

2